In [3]:
from htools.htools import hdir
import numpy as np
import pandas as pd
import re

## tdir

In [39]:
def tdir(obj, methods=False, **kwargs):
    out = dict()
    for k, v in hdir(obj, **kwargs).items():
        if v == 'attribute':
            out[k] = type(getattr(obj, k))
            
        # Only methods that don't require additional arguments are added.
        elif methods and v == 'method':
            try:
                out[k] = type(getattr(obj, k)())
            except TypeError:
                continue
        
    return out    

In [32]:
tdir(6)

{'denominator': int, 'imag': int, 'numerator': int, 'real': int}

In [33]:
tdir(0, magics=True)

{'__doc__': str,
 'denominator': int,
 'imag': int,
 'numerator': int,
 'real': int}

In [35]:
tdir(0, True)

{'bit_length': int,
 'conjugate': int,
 'denominator': int,
 'imag': int,
 'numerator': int,
 'real': int}

In [37]:
a = 0
a.conjugate()

0

## hdir (aybe change to cdir for callable or mdir for methoddir or adir for attrdir)

In [51]:
a = [1, 2, 3]
s = 'abc'
t = ('a', 'b')
d = {'a': 1, 'b': 2}
n = np.arange(5)

items = [a, s, t, d, n]

In [52]:
for item in items:
    print(hdir(item), end='\n\n')

{'append': 'method', 'clear': 'method', 'copy': 'method', 'count': 'method', 'extend': 'method', 'index': 'method', 'insert': 'method', 'pop': 'method', 'remove': 'method', 'reverse': 'method', 'sort': 'method'}

{'capitalize': 'method', 'casefold': 'method', 'center': 'method', 'count': 'method', 'encode': 'method', 'endswith': 'method', 'expandtabs': 'method', 'find': 'method', 'format': 'method', 'format_map': 'method', 'index': 'method', 'isalnum': 'method', 'isalpha': 'method', 'isascii': 'method', 'isdecimal': 'method', 'isdigit': 'method', 'isidentifier': 'method', 'islower': 'method', 'isnumeric': 'method', 'isprintable': 'method', 'isspace': 'method', 'istitle': 'method', 'isupper': 'method', 'join': 'method', 'ljust': 'method', 'lower': 'method', 'lstrip': 'method', 'maketrans': 'method', 'partition': 'method', 'replace': 'method', 'rfind': 'method', 'rindex': 'method', 'rjust': 'method', 'rpartition': 'method', 'rsplit': 'method', 'rstrip': 'method', 'split': 'method', 'spli

In [21]:
def is_internal(name):
    return bool(re.match('_[^_]', name))

In [22]:
[is_internal(attr) for attr in ['_add', '__add__']]

[True, False]

In [45]:
def mdir(obj, magics=False, internals=False):
    out = {attr: ('method' if callable(getattr(obj, attr)) else 'attribute')
              for attr in dir(obj)}
    if magics and internals:
        return out
    
    def keep_attr(attr, magics, internals):
        return (magics or not attr.startswith('__')) and \
               (internals or not re.match('_[^_]', attr))

    return {k: v for k, v in out.items() if keep_attr(k, magics, internals)}

In [101]:
def hdir_new(obj, magics=False, internals=False):
    output = dict()
    for attr in dir(obj):
        # Exclude magics or internals if specified.
        if (not magics and attr.startswith('__')) or \
           (not internals and re.match('_[^_]', attr)):
            continue
            
        # Handle rare cases where...
        try:
            is_method = callable(getattr(obj, attr))
        except Exception:
            continue
        
        # Update output to specify whether attr is callable.
        if is_method:
            output[attr] = 'method'
        else:
            output[attr] = 'attribute'
    return output

In [104]:
# Testing alternative to try/except block.

# invoked = getattr(obj, attr, None)
# if not invoked:
#     continue
    
# if callable(invoked):
#     output[attr] = 'method'
# else:
#     output[attr] = 'attribute'

In [87]:
class Block:
    
    def __init__(self, x):
        self.x = x
        
    def a(self):
        return 'a'
    
    def _b(self):
        return '_b'
    
    def __str__(self):
        return 'Block'

In [105]:
b = Block(4)

In [108]:
for magic_bool in (True, False):
    for internal_bool in (True, False):
        args = (magic_bool, internal_bool)
        print(args)
        print(hdir(n, *args) == mdir(n, *args) == hdir_new(n, *args))

(True, True)
True
(True, False)
True
(False, True)
True
(False, False)
True


In [91]:
df = pd.DataFrame(np.random.randint(10, size=(5, 3)))
df

,0,1,2
0,6,0,0
1,2,3,8
2,6,2,4
3,1,6,4
4,6,7,4


In [107]:
hdir_new(df, magics=True)

/Users/hmamin/anaconda3/envs/deeplearning/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: DataFrame.ftypes is deprecated and will be removed in a future version. Use DataFrame.dtypes instead.
  # This is added back by InteractiveShellApp.init_path()
/Users/hmamin/anaconda3/envs/deeplearning/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


{'T': 'attribute',
 '__abs__': 'method',
 '__add__': 'method',
 '__and__': 'method',
 '__array__': 'method',
 '__array_priority__': 'attribute',
 '__array_wrap__': 'method',
 '__bool__': 'method',
 '__class__': 'method',
 '__contains__': 'method',
 '__copy__': 'method',
 '__deepcopy__': 'method',
 '__delattr__': 'method',
 '__delitem__': 'method',
 '__dict__': 'attribute',
 '__dir__': 'method',
 '__div__': 'method',
 '__doc__': 'attribute',
 '__eq__': 'method',
 '__finalize__': 'method',
 '__floordiv__': 'method',
 '__format__': 'method',
 '__ge__': 'method',
 '__getattr__': 'method',
 '__getattribute__': 'method',
 '__getitem__': 'method',
 '__getstate__': 'method',
 '__gt__': 'method',
 '__hash__': 'method',
 '__iadd__': 'method',
 '__iand__': 'method',
 '__ifloordiv__': 'method',
 '__imod__': 'method',
 '__imul__': 'method',
 '__init__': 'method',
 '__init_subclass__': 'method',
 '__invert__': 'method',
 '__ior__': 'method',
 '__ipow__': 'method',
 '__isub__': 'method',
 '__iter__':

In [116]:
for flag1 in (True, False):
    for flag2 in (True, False):
        print(flag1, flag2)
        print(hdir_new(b, flag1, flag2), end='\n\n')

True True
{'__class__': 'method', '__delattr__': 'method', '__dict__': 'attribute', '__dir__': 'method', '__doc__': 'attribute', '__eq__': 'method', '__format__': 'method', '__ge__': 'method', '__getattribute__': 'method', '__gt__': 'method', '__hash__': 'method', '__init__': 'method', '__init_subclass__': 'method', '__le__': 'method', '__lt__': 'method', '__module__': 'attribute', '__ne__': 'method', '__new__': 'method', '__reduce__': 'method', '__reduce_ex__': 'method', '__repr__': 'method', '__setattr__': 'method', '__sizeof__': 'method', '__str__': 'method', '__subclasshook__': 'method', '__weakref__': 'attribute', '_b': 'method', 'a': 'method', 'x': 'attribute'}

True False
{'__class__': 'method', '__delattr__': 'method', '__dict__': 'attribute', '__dir__': 'method', '__doc__': 'attribute', '__eq__': 'method', '__format__': 'method', '__ge__': 'method', '__getattribute__': 'method', '__gt__': 'method', '__hash__': 'method', '__init__': 'method', '__init_subclass__': 'method', '__l

In [114]:
hdir_new(b, True, True)

{'__class__': 'method',
 '__delattr__': 'method',
 '__dict__': 'attribute',
 '__dir__': 'method',
 '__doc__': 'attribute',
 '__eq__': 'method',
 '__format__': 'method',
 '__ge__': 'method',
 '__getattribute__': 'method',
 '__gt__': 'method',
 '__hash__': 'method',
 '__init__': 'method',
 '__init_subclass__': 'method',
 '__le__': 'method',
 '__lt__': 'method',
 '__module__': 'attribute',
 '__ne__': 'method',
 '__new__': 'method',
 '__reduce__': 'method',
 '__reduce_ex__': 'method',
 '__repr__': 'method',
 '__setattr__': 'method',
 '__sizeof__': 'method',
 '__str__': 'method',
 '__subclasshook__': 'method',
 '__weakref__': 'attribute',
 '_b': 'method',
 'a': 'method',
 'x': 'attribute'}

In [111]:
%%timeit -n 50 -r 50
hdir_new(n)

157 µs ± 51.5 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [95]:
def attr_filter_factory(magics, internals):
    
    def filter_(attr):
        if attr.startswith('__'):
            if not magics: 
                return False
            return True
        
        if attr.startswith('_'):
            if not internals:
                return False
            return True
        
        return True
    
    return filter_

In [50]:
filters = dict(dummy_filter=attr_filter_factory(True, True),
               no_magic_filter=attr_filter_factory(False, True),
               no_internal_filter=attr_filter_factory(True, False),
               no_magic_or_internal_filter=attr_filter_factory(False, False))

In [51]:
for name, f in filters.items():
    print(f'{name} \n__add__: {f("__add__")} \n_recurse: {f("_internal")}\n')

dummy_filter 
__add__: True 
_recurse: True

no_magic_filter 
__add__: False 
_recurse: True

no_internal_filter 
__add__: True 
_recurse: False

no_magic_or_internal_filter 
__add__: False 
_recurse: False



In [11]:
mdir(a)

{'__add__': 'method',
 '__class__': 'method',
 '__contains__': 'method',
 '__delattr__': 'method',
 '__delitem__': 'method',
 '__dir__': 'method',
 '__doc__': 'attr',
 '__eq__': 'method',
 '__format__': 'method',
 '__ge__': 'method',
 '__getattribute__': 'method',
 '__getitem__': 'method',
 '__gt__': 'method',
 '__hash__': 'attr',
 '__iadd__': 'method',
 '__imul__': 'method',
 '__init__': 'method',
 '__init_subclass__': 'method',
 '__iter__': 'method',
 '__le__': 'method',
 '__len__': 'method',
 '__lt__': 'method',
 '__mul__': 'method',
 '__ne__': 'method',
 '__new__': 'method',
 '__reduce__': 'method',
 '__reduce_ex__': 'method',
 '__repr__': 'method',
 '__reversed__': 'method',
 '__rmul__': 'method',
 '__setattr__': 'method',
 '__setitem__': 'method',
 '__sizeof__': 'method',
 '__str__': 'method',
 '__subclasshook__': 'method',
 'append': 'method',
 'clear': 'method',
 'copy': 'method',
 'count': 'method',
 'extend': 'method',
 'index': 'method',
 'insert': 'method',
 'pop': 'method'

## Example for htimer docstring

In [99]:
import time

@htimer
def count_to(x):
    for i in range(x):
        time.sleep(.5)
    print(x)

In [100]:
count_to(10)

10

[TIMER]: function <count_to> executed in roughly 5.0365 seconds (conservatively).

